In [127]:
# Imports
import numpy as np #Mathematical tools
import matplotlib.pyplot as plt #Plots charts
import pandas as pd #Import and manage data sets
from sklearn.neural_network import MLPClassifier 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
# Preprocessing library
from sklearn.preprocessing import Imputer
# Categorical data encoder library
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Splitting the data set into training and testing sets library
from sklearn.model_selection import train_test_split
#Feature scaling library
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [128]:
# Imports part 2

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [129]:
# Functions part 1

def best_output(row):
    # se o downloadTime é diferente de 100 para T2 e T3, ambos os targets completam o download
    if ((row.downloadTimeT2!=100)&(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é menor
        if (row.downloadTimeT2<=row.downloadTimeT3):
            return 0
        else:
            return 1
    
    # se o downloadTime é diferente de 100 apenas para um dos targets, só um deles completa o download
    elif ((row.downloadTimeT2!=100)|(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é diferente de 100
        if (row.downloadTimeT2!=100):
            return 0
        else:
            return 1
        
    # se o downloadTime é igual a 100 para T2 e T3, ambos os targets não completam o download
    elif ((row.downloadTimeT2==100)&(row.downloadTimeT3==100)):
        # o melhor output será aquele para o qual o rxBytes é maior
        if (row.rxBytesT2>=row.rxBytesT3):
            return 0
        else:
            return 1
        
def download_complete(row, column):
    if (row[column]==0):
        if(row.downloadTimeT2!=100.0):
            return 1
        else:
            return 0
    else:
        if(row.downloadTimeT3!=100.0):
            return 1
        else: 
            return 0

        
def download_time(row, column):
    if ((row[column]==0)&(row.downloadTimeT2<100)):
        return row.downloadTimeT2
    elif ((row[column]==1)&(row.downloadTimeT3<100)):
        return row.downloadTimeT3

    
def throughput(row, column):
    if (row[column]==0):
        tp = (row.rxBytesT2/row.downloadTimeT2)*8/1e6
        return tp
    else:
        tp = (row.rxBytesT3/row.downloadTimeT3)*8/1e6
        return tp
    
def optimum_choice(row, column):
    if (row[column]==row.best_output):
        return 1
    else:
        return 0
    
def a2a4rsrp(row):
    if (row.rsrp2>=row.rsrp3):
        return 0
    else:
        return 1

In [130]:
# Functions part 2
def percentageOfErrors(y_real, y_predicted):
    p= ((y_predicted != y_real).sum()/ y_real.size)*100
    return p 

def applyingClassifier (classifier,x_test, x_train, y_test, y_train, indexes):
    classifier.fit(x_train, y_train)
    # Predicting the test set results
    y_pred = classifier.predict(x_test)
    # Calculating statistics like average download time
    stats=calculatingStatistics(indexes=indexes, choice=y_pred)
    
    # Evaluation of the model
    # Making the confusion matrix
    #cm = confusion_matrix(y_test, y_pred)
    p=percentageOfErrors(y_real=y_test, y_predicted=y_pred)
    s=np.array([ stats[0], stats[1], stats[2], 100 - p])
    print(s)
    return s

def gettingClassifierConfiguration(classifier, param_test,x_train, y_train ):
    conf=GridSearchCV(estimator=classifier, param_grid=param_test, scoring='accuracy', cv=10, n_jobs=-1)
    conf.fit(x_train, y_train)
    print(conf.best_estimator_)
    return conf.best_estimator_

def gettingAllClassifiersConfigurations(x_train, y_train, randomState):
    # All test parameters
    param_test_knn = [{'n_neighbors':[(i) for i in range(5,31)], 'p':[1,2],'weights':['uniform', 'distance'],
                  'algorithm':['auto', 'brute']},{'n_neighbors':[(i) for i in range(5,31)], 'p':[1,2],
                 'weights':['uniform', 'distance'],
                  'algorithm':['ball_tree', 'kd_tree'], 'leaf_size':[10,20,30,40,50,60,70, 80,90,100]} ]
    param_test_svm =[{'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'degree':[(i) for i in range(1,21)],'kernel':['poly'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
                 {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['linear'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
              {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['rbf', 'sigmoid'],'gamma':[1,0.5,0.1, 0.01,0.001, 0.0001, 0.0000001],
                 'decision_function_shape':['ovo','ovr'], 'shrinking':[True, False]}] 
    # Creating the DT classifier parameters
    param_test_dt=[{'criterion': ['gini', 'entropy'], 'splitter':['best', 'random'], 
                    'max_features':[None,'auto', 'sqrt', 'log2'], 'class_weight':['balanced', None]}]    
   
    # Creating the RF classifier parameters
    param_test_rf = {'n_estimators':[(i) for i in range(5,31)], 'criterion': ['gini', 'entropy'], 
                     'max_features':[None,'auto', 'sqrt', 'log2'], 'class_weight':['balanced', None], 
                     'verbose':[0], 'n_jobs':[-1], 'bootstrap':[True, False]}
    
    hl=[[30, 20, 10], [10, 10, 10], [10, 20, 30], [10, 10], [10, 20], [20, 10], [60, 40, 20], [20, 20, 20], [20, 40, 60], [20, 20], [20, 40], [40, 20], [90, 60, 30], [30, 30, 30], [30, 60, 90], [30, 30], [30, 60], [60, 30], [120, 80, 40], [40, 40, 40], [40, 80, 120], [40, 40], [40, 80], [80, 40], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100]]
   
    param_test_nn = [{'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' :['lbfgs'],
                'learning_rate' : ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': hl,
                     'max_iter':[1500]},{'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' :['sgd', 'adam'],
                'learning_rate' : ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': hl,
                     'max_iter':[1500], 'early_stopping':[True, False], 'validation_fraction':[0.1,0.15,0.2],'shuffle':[True, False],
                                         'learning_rate_init':[0.001,0.01, 0.1]}]
    # All classifiers
    classifier_knn=KNeighborsClassifier(metric = 'minkowski')
    classifier_svm=SVC(random_state=randomState)
    classifier_dt=DecisionTreeClassifier(random_state=randomState)
    classifier_rf=RandomForestClassifier(random_state=randomState)
    classifier_nn=MLPClassifier(random_state=randomState)
    
    #Finding the best configurations
    
    print("Best parameters for NN Classifier")
    conf_nn=gettingClassifierConfiguration(classifier=classifier_nn, param_test=param_test_nn,
                                           x_train=x_train, y_train=y_train)
    
    print("Best parameters for KNN Classifier")
    conf_knn=gettingClassifierConfiguration(classifier=classifier_knn, param_test=param_test_knn, 
                                            x_train=x_train, y_train=y_train)
    
    print("Best parameters for SVM Classifier")
    conf_svm=gettingClassifierConfiguration(classifier=classifier_svm, param_test=param_test_svm,
                                            x_train=x_train, y_train=y_train)
    print("Best parameters for DT Classifier")
    conf_dt=gettingClassifierConfiguration(classifier=classifier_dt, param_test=param_test_dt,
                                           x_train=x_train, y_train=y_train)
    print("Best parameters for RF Classifier")
    conf_rf=gettingClassifierConfiguration(classifier=classifier_rf, param_test=param_test_rf,
                                           x_train=x_train, y_train=y_train)
    
    
    conf=[conf_knn,conf_svm, conf_dt, conf_rf, conf_nn]
    return conf

def theBestScore(oldBest, candidate):
    if oldBest>candidate:
        return oldBest
    else:
        return candidate    
def theWorstScore(oldWorst, candidate):
    if oldWorst<candidate:
        return oldWorst
    else:
        return candidate
    
def addArray(array1, array2):
    answer=[a1 + a2 for a1, a2 in zip(array1, array2)]
    return answer
    
    
def applyingAllClassifiers (classifiers,numberOfRandomStates,x, y):   
    # Initializing the variables
    # Averages
    p_svm=0
    p_dt=0
    p_rf=0
    p_knn=0
    p_nn=0
    # Worst cases 
    w_knn=100
    w_svm=100
    w_rf=100
    w_dt=100
    w_nn=100
    # Best cases
    b_svm=0
    b_dt=0
    b_rf=0
    b_knn=0
    b_nn=0
    
    #Other stats
    stats_knn=np.array([0,0,0,0])
    stats_svm=np.array([0,0,0,0])
    stats_dt=np.array([0,0,0,0])
    stats_rf=np.array([0,0,0,0])
    stats_nn=np.array([0,0,0,0])
    
    for (i) in range(0,numberOfRandomStates):
        #r1=np.random.randint(1,10001)
        #r2=np.random.randint(1,10001)
        r1=i
        r2=i
        # Splitting test data and train data
        x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = r1)
        indexes=x_test.axes[0]
        indexes=indexes.values
        # Feature scaling (normalizing or stadardization of the scales)
        # Helps the conversion of the algorithm
        sc_X = StandardScaler()
        x_train = sc_X.fit_transform(x_train)
        x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit        
        print("{}: RandomState r1={} and r2={}".format(i, r1,r2))
        # Fitting the KNN to the training set
        stats=applyingClassifier(indexes=indexes,classifier=classifiers[0],x_test=x_test, x_train=x_train, y_test=y_test, y_train= y_train)
        p=stats[3]
        stats_knn=addArray(stats_knn, stats)
        p_knn=p_knn+p
        w_knn=theWorstScore(oldWorst=w_knn, candidate=p)
        b_knn=theBestScore(oldBest=b_knn, candidate=p)
        # Fitting the SVM to the training set
        classifiers[1].random_state=r2
        stats=applyingClassifier(indexes=indexes,classifier=classifiers[1],x_test=x_test, x_train=x_train, y_test=y_test, y_train= y_train)
        p=stats[3]
        stats_svm=addArray(stats_svm, stats)
        p_svm=p_svm+p
        w_svm=theWorstScore(oldWorst=w_svm, candidate=p)
        b_svm=theBestScore(oldBest=b_svm, candidate=p)
        # Fitting the DT to the training set
        classifiers[2].random_state=r2
        stats=applyingClassifier(indexes=indexes,classifier=classifiers[2],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        p=stats[3]
        stats_dt=addArray(stats_dt, stats)
        p_dt=p_dt+p
        w_dt=theWorstScore(oldWorst=w_dt, candidate=p)
        b_dt=theBestScore(oldBest=b_dt, candidate=p)
        # Fitting the RF to the training set
        classifiers[3].random_state=r2
        stats=applyingClassifier(indexes=indexes,classifier=classifiers[3],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        p=stats[3]
        stats_rf=addArray(stats_rf, stats)
        p_rf=p_rf+p
        w_rf=theWorstScore(oldWorst=w_rf, candidate=p)
        b_rf=theBestScore(oldBest=b_rf, candidate=p)
        # Fitting the NN to the training set
        classifiers[4].random_state=r2
        stats=applyingClassifier(indexes=indexes,classifier=classifiers[4],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        p=stats[3]
        stats_nn=addArray(stats_nn, stats)
        p_nn=p_nn+p
        w_nn=theWorstScore(oldWorst=w_nn, candidate=p)
        b_nn=theBestScore(oldBest=b_nn, candidate=p)
        
    stats_knn=[elem / numberOfRandomStates for elem in stats_knn]
    stats_svm=[elem / numberOfRandomStates for elem in stats_svm]
    stats_dt=[elem / numberOfRandomStates for elem in stats_dt]
    stats_rf=[elem / numberOfRandomStates for elem in stats_rf]
    stats_nn=[elem / numberOfRandomStates for elem in stats_nn]
    
    stats_knn.extend([b_knn, w_knn])
    stats_svm.extend([b_svm, w_svm])
    stats_dt.extend([b_dt, w_dt])
    stats_rf.extend([b_rf, w_rf])
    stats_nn.extend([b_nn, w_nn])
    stats=np.array([stats_knn,stats_svm, stats_dt, stats_rf, stats_nn])
    
    tp=throughput(rxBytes=stats[:,1], downloadTime=stats[:,0])
    s=pd.DataFrame({'Algorithm':['KNN', 'SVM', 'DT', 'RF', 'NN'],'Average Score':stats[:,3], 'Best score': stats[:,4], 'Worst score': stats[:,5],'Downloads Completed':stats[:,2],'Download Time':stats[:,0], 'Received Bytes': stats[:,1], 'Throughput': tp})
    return s   
    

In [131]:
# Functions part 3
def calculatingStatistics(indexes, choice):
    downloadTime=[]
    rxBytes=[]
    downloadsCompleted=0
    for i in range(0, 200):
        index=indexes[i]
        if(choice[i]==0):
            downloadTime.extend([data.downloadTimeT2[index]])
            rxBytes.extend([data.rxBytesT2[index]])
        else:
            downloadTime.extend([data.downloadTimeT3[index]])
            rxBytes.extend([data.rxBytesT3[index]])
        if(downloadTime[i]!=100):
            downloadsCompleted=downloadsCompleted+1
    meanDownloadTime=np.mean(downloadTime) 
    meanRxBytes=np.mean(rxBytes)
    downloadsCompleted=downloadsCompleted/2 # Divided by 200 and multiplied by 100
    return[meanDownloadTime, meanRxBytes, downloadsCompleted]

float_formatter = lambda num: "%.2f" % num
np.set_printoptions(formatter={'float_kind':float_formatter})
pd.options.display.float_format = float_formatter

def throughput(rxBytes, downloadTime):
    tp=[i/j for i,j in zip(rxBytes, downloadTime)]
    return tp



# Sem shadowing

In [132]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OkumuraHata_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OkumuraHata_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

# Splitting the data between independent variables and dependent variable
y = data['best_output']
x = data[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]

# Splitting test data and train data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
# Feature scaling (normalizing or stadardization of the scales)
# Helps the conversion of the algorithm
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit


In [133]:
classifier_knn=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=1,
           weights='uniform')

classifier_svm=SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

classifier_dt=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')


classifier_rf=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

classifier_nn=MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[10,10,10], learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

conf=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]

In [134]:
# Comment this line if you want to use the classifiers above
#conf=gettingAllClassifiersConfigurations(x_train=x_train, y_train=y_train, randomState=0)

# Setting the classifiers
classifier_knn=conf[0]
classifier_svm=conf[1]
classifier_dt=conf[2]
classifier_rf=conf[3]
classifier_nn=conf[4]


classifiers=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]

In [135]:
# Applying the classifiers
maxRange=100
stats = applyingAllClassifiers(numberOfRandomStates=maxRange, x=x, y=y, classifiers=classifiers)


0: RandomState r1=0 and r2=0
[39.82 15728640.00 100.00 100.00]
[39.85 15728640.00 100.00 99.50]
[39.85 15728640.00 100.00 99.50]
[39.85 15728640.00 100.00 99.50]
[39.85 15728640.00 100.00 99.50]
1: RandomState r1=1 and r2=1
[40.17 15728449.44 99.50 99.50]
[39.91 15728640.00 100.00 100.00]
[39.96 15728640.00 100.00 99.50]
[39.96 15728640.00 100.00 99.50]
[39.91 15728640.00 100.00 100.00]
2: RandomState r1=2 and r2=2
[40.49 15724218.96 99.00 99.00]
[40.23 15724409.52 99.50 99.50]
[40.23 15724409.52 99.50 99.50]
[39.98 15728640.00 100.00 100.00]
[40.49 15724218.96 99.00 99.00]
3: RandomState r1=3 and r2=3
[40.28 15724409.52 99.50 99.50]
[40.28 15724409.52 99.50 99.50]
[40.33 15724409.52 99.50 99.00]
[40.28 15724409.52 99.50 99.50]
[40.28 15724409.52 99.50 99.50]
4: RandomState r1=4 and r2=4
[39.87 15728640.00 100.00 100.00]
[39.90 15728640.00 100.00 99.50]
[39.90 15728640.00 100.00 99.50]
[39.90 15728640.00 100.00 99.50]
[39.90 15728640.00 100.00 99.50]
5: RandomState r1=5 and r2=5
[40.39

[39.95 15728640.00 100.00 99.50]
[39.90 15728640.00 100.00 100.00]
43: RandomState r1=43 and r2=43
[40.14 15724409.52 99.50 99.50]
[40.14 15724409.52 99.50 99.50]
[40.14 15724409.52 99.50 99.50]
[40.14 15724409.52 99.50 99.50]
[40.14 15724409.52 99.50 99.50]
44: RandomState r1=44 and r2=44
[39.80 15728640.00 100.00 100.00]
[39.80 15728640.00 100.00 100.00]
[39.85 15728640.00 100.00 99.50]
[39.80 15728640.00 100.00 100.00]
[39.80 15728640.00 100.00 100.00]
45: RandomState r1=45 and r2=45
[39.93 15728640.00 100.00 100.00]
[39.93 15728640.00 100.00 100.00]
[39.93 15728640.00 100.00 100.00]
[39.93 15728640.00 100.00 100.00]
[39.93 15728640.00 100.00 100.00]
46: RandomState r1=46 and r2=46
[39.95 15728640.00 100.00 100.00]
[39.95 15728640.00 100.00 100.00]
[40.00 15728640.00 100.00 99.50]
[40.00 15728640.00 100.00 99.50]
[39.95 15728640.00 100.00 100.00]
47: RandomState r1=47 and r2=47
[39.84 15728640.00 100.00 100.00]
[39.84 15728640.00 100.00 100.00]
[39.84 15728640.00 100.00 100.00]
[39.

[39.99 15728640.00 100.00 99.50]
[39.99 15728640.00 100.00 99.50]
85: RandomState r1=85 and r2=85
[40.05 15728640.00 100.00 100.00]
[40.05 15728640.00 100.00 100.00]
[40.10 15728640.00 100.00 99.50]
[40.10 15728640.00 100.00 99.50]
[40.05 15728640.00 100.00 100.00]
86: RandomState r1=86 and r2=86
[40.07 15728640.00 100.00 100.00]
[40.07 15728640.00 100.00 100.00]
[40.07 15728640.00 100.00 100.00]
[40.07 15728640.00 100.00 100.00]
[40.07 15728640.00 100.00 100.00]
87: RandomState r1=87 and r2=87
[39.83 15728640.00 100.00 100.00]
[39.83 15728640.00 100.00 100.00]
[39.88 15728640.00 100.00 99.50]
[39.83 15728640.00 100.00 100.00]
[39.83 15728640.00 100.00 100.00]
88: RandomState r1=88 and r2=88
[39.86 15728640.00 100.00 100.00]
[39.86 15728640.00 100.00 100.00]
[39.91 15728640.00 100.00 99.50]
[39.91 15728640.00 100.00 99.50]
[39.86 15728640.00 100.00 100.00]
89: RandomState r1=89 and r2=89
[39.95 15728640.00 100.00 100.00]
[39.95 15728640.00 100.00 100.00]
[40.00 15728640.00 100.00 99.50

In [136]:
print('Results without shadowing:')
print(stats)
stats

Results without shadowing:
  Algorithm  Average Score  Best score  Download Time  Downloads Completed  \
0       KNN          99.67      100.00          40.01                99.69   
1       SVM          99.77      100.00          39.95                99.80   
2        DT          99.59      100.00          39.96                99.81   
3        RF          99.64      100.00          39.96                99.80   
4        NN          99.75      100.00          39.95                99.80   

   Received Bytes  Throughput  Worst score  
0     15727167.80   393118.42        98.00  
1     15727373.38   393703.99        98.00  
2     15727415.68   393545.61        98.00  
3     15727494.57   393571.53        98.00  
4     15727413.77   393679.54        98.00  


,Algorithm,Average Score,Best score,Download Time,Downloads Completed,Received Bytes,Throughput,Worst score
0,KNN,99.67,100.00,40.01,99.69,15727167.80,393118.42,98.00
1,SVM,99.77,100.00,39.95,99.80,15727373.38,393703.99,98.00
2,DT,99.59,100.00,39.96,99.81,15727415.68,393545.61,98.00
3,RF,99.64,100.00,39.96,99.80,15727494.57,393571.53,98.00
4,NN,99.75,100.00,39.95,99.80,15727413.77,393679.54,98.00


# Com shadowing

In [137]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OhBuildings_ComShadowing_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OhBuildings_ComShadowing_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

# Splitting the data between independent variables and dependent variable
y = data['best_output']
x = data[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]

# Splitting test data and train data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
# Feature scaling (normalizing or stadardization of the scales)
# Helps the conversion of the algorithm
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit


In [138]:
classifier_nn=MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[10, 10, 10], learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

classifier_knn=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=21, p=2,
           weights='uniform')


classifier_svm=SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

classifier_dt=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

classifier_rf=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=9, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

conf=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]


In [139]:
# Comment this line if you want to use the classifiers above
#conf=gettingAllClassifiersConfigurations(x_train=x_train, y_train=y_train, randomState=0)

# Setting the classifiers
classifier_knn=conf[0]
classifier_svm=conf[1]
classifier_dt=conf[2]
classifier_rf=conf[3]
classifier_nn=conf[4]


classifiers=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]

In [140]:
# Applying the classifiers
maxRange=100
stats = applyingAllClassifiers(numberOfRandomStates=maxRange, x=x, y=y, classifiers=classifiers)

0: RandomState r1=0 and r2=0
[71.74 11230944.88 46.00 60.82]
[60.03 12937855.80 65.00 83.29]
[68.91 11561529.68 50.50 64.11]
[71.23 11267711.92 46.50 62.47]
[62.45 12607480.96 60.50 79.45]
1: RandomState r1=1 and r2=1
[70.88 11809772.76 48.50 65.21]
[61.59 12993114.36 62.50 84.11]
[75.80 11112026.68 39.50 57.26]
[71.86 11768741.64 46.50 65.48]
[61.25 13213897.88 64.00 82.47]
2: RandomState r1=2 and r2=2
[68.63 11761990.52 52.00 66.03]
[63.45 12501497.20 59.50 79.73]
[70.56 11217806.32 48.00 66.30]
[69.75 11682826.32 50.00 66.85]
[62.79 12824253.80 60.50 82.47]
3: RandomState r1=3 and r2=3
[70.74 11387212.76 47.50 60.55]
[60.58 13088358.60 64.50 79.45]
[69.02 11764194.08 51.00 61.64]
[66.98 11729036.08 54.00 64.38]
[60.61 13123361.68 64.00 82.19]
4: RandomState r1=4 and r2=4
[68.69 11618648.64 49.50 60.82]
[59.83 13225583.24 64.00 80.82]
[68.32 11812612.88 50.50 61.92]
[67.39 12096432.76 51.50 65.21]
[60.23 13165971.68 63.00 81.10]
5: RandomState r1=5 and r2=5
[69.59 11483724.28 48.00 6

[61.87 13013408.36 61.00 82.47]
43: RandomState r1=43 and r2=43
[69.08 11760897.28 52.00 61.37]
[61.95 13081398.92 62.50 81.64]
[73.12 11266833.84 44.50 59.45]
[69.64 11926092.64 50.50 65.75]
[60.78 13184925.72 64.50 81.37]
44: RandomState r1=44 and r2=44
[68.14 11862573.52 51.50 66.85]
[65.18 12646695.96 56.50 79.73]
[71.92 11517501.76 45.50 63.56]
[69.63 11833973.48 49.50 66.85]
[63.71 12819729.08 59.00 79.45]
45: RandomState r1=45 and r2=45
[71.74 11805643.92 46.50 61.64]
[64.73 13041174.60 56.50 79.18]
[72.53 11580412.72 44.50 62.74]
[72.24 11622689.40 45.00 64.66]
[63.49 13046015.28 59.00 79.73]
46: RandomState r1=46 and r2=46
[69.91 11858748.76 49.50 67.12]
[62.77 13170506.80 60.50 79.45]
[71.65 11398642.68 45.50 63.56]
[69.81 11791091.24 48.00 67.67]
[62.80 13066234.00 60.00 83.84]
47: RandomState r1=47 and r2=47
[66.60 12015353.76 55.00 66.30]
[60.48 13076854.24 64.50 80.27]
[69.52 11563091.20 50.00 58.36]
[69.17 11777947.76 50.50 61.10]
[63.90 12303065.88 58.50 74.25]
48: Rand

[71.76 11778054.88 45.50 65.48]
[62.71 13157648.56 60.50 84.38]
86: RandomState r1=86 and r2=86
[71.36 11690788.56 46.50 62.47]
[61.91 13042440.48 62.50 82.19]
[74.36 11225472.28 41.50 60.00]
[71.04 11821260.60 46.50 62.74]
[61.08 13062707.56 63.00 84.11]
87: RandomState r1=87 and r2=87
[69.95 11602269.72 47.50 63.01]
[61.39 12972840.04 62.00 80.55]
[71.47 11470958.12 46.00 61.64]
[70.67 11499343.56 47.00 64.11]
[61.26 13020710.92 62.00 82.74]
88: RandomState r1=88 and r2=88
[69.90 11513176.60 48.00 61.64]
[61.11 12963567.92 62.50 80.00]
[70.87 11470140.00 46.00 64.66]
[68.08 11856381.00 51.00 66.30]
[60.69 12904788.68 63.00 79.73]
89: RandomState r1=89 and r2=89
[71.53 11425729.84 46.00 70.96]
[66.49 12396054.68 55.00 82.74]
[70.42 11500586.24 48.00 65.75]
[75.05 10964743.36 39.50 64.66]
[65.15 12597536.36 56.50 86.30]
90: RandomState r1=90 and r2=90
[70.11 11547078.92 49.00 61.64]
[62.65 12786427.12 61.00 78.90]
[70.57 11592898.56 48.50 63.01]
[70.30 11519258.20 48.00 64.38]
[61.62 1

In [141]:
print('Results with shadowing:')
print(stats)
stats

Results with shadowing:
  Algorithm  Average Score  Best score  Download Time  Downloads Completed  \
0       KNN          64.39       70.96          69.83                48.92   
1       SVM          81.49       84.66          61.77                62.10   
2        DT          62.69       69.04          71.28                46.52   
3        RF          65.16       71.51          70.51                47.66   
4        NN          82.02       87.67          61.89                61.70   

   Received Bytes  Throughput  Worst score  
0     11761527.44   168419.27        58.36  
1     12999611.22   210448.16        76.44  
2     11584194.69   162507.91        56.71  
3     11726021.95   166309.19        59.73  
4     12961937.19   209421.91        74.25  


,Algorithm,Average Score,Best score,Download Time,Downloads Completed,Received Bytes,Throughput,Worst score
0,KNN,64.39,70.96,69.83,48.92,11761527.44,168419.27,58.36
1,SVM,81.49,84.66,61.77,62.10,12999611.22,210448.16,76.44
2,DT,62.69,69.04,71.28,46.52,11584194.69,162507.91,56.71
3,RF,65.16,71.51,70.51,47.66,11726021.95,166309.19,59.73
4,NN,82.02,87.67,61.89,61.70,12961937.19,209421.91,74.25
